In [2]:
import sys
sys.path.append('/Users/gscerberus/Earthquake_Damage_Predicton')

In [3]:
import os
os.chdir('/Users/gscerberus/Earthquake_Damage_Predicton')

In [13]:
from src.hyperparameter_tuning import Tuning
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier



In [5]:
df = pd.read_csv('datasets/final_version.csv')

We will divide our dataset into 3 sets, training, validation and test sets. We will perform hyperparameter tuning on our validation set, to see what are the most appropriate hyperparameter values we can choose, so our models perform the best at.

In [6]:
# Will create 60-20-20 split
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


/Users/gscerberus/Earthquake_Damage_Predicton/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
train.shape, validate.shape, test.shape

((457263, 40), (152421, 40), (152422, 40))

In [8]:
#storing them as csv files, to be reused later
train.to_csv('datasets/train_data.csv', index=False)
test.to_csv('datasets/test_data.csv', index=False)
validate.to_csv('datasets/validate_data.csv', index=False)

In [10]:
X = validate.drop(columns=['severe_damage'])
y = validate['severe_damage']

In [11]:
# Example parameter grid for each algorithm
param_grid_lr = {'C' : np.logspace(-4, 4, 20)}

param_grid_dt = {'criterion':['gini','entropy'],
                 'max_depth': range(1,10)}

param_grid_rf = {'n_estimators': range(1,300,100), 'max_depth': range(1,30,10)}

param_grid_sv = {'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1]}

In [20]:
# LOGISTIC REGRESSION
lr_model = LogisticRegression()
tuner = Tuning(lr_model, param_grid_lr)
best_params_lr, best_score_lr = tuner.tune(X, y)

/Users/gscerberus/Earthquake_Damage_Predicton/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/gscerberus/Earthquake_Damage_Predicton/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

In [21]:
print(f"Best hyperparameters for logistic regression: {best_params_lr},   {best_score_lr}") #0.0001 learning rate?

Best hyperparameters for logistic regression: {'C': 0.08858667904100823},   0.7192578576538844


In [22]:
# DECISION TREES
dt_model = DecisionTreeClassifier()
tuner = Tuning(dt_model, param_grid_dt)
best_params_dt, best_score_dt = tuner.tune(X, y)

In [23]:
print(f"Best hyperparameters for decision trees: {best_params_dt},   {best_score_dt}")

Best hyperparameters for decision trees: {'criterion': 'gini', 'max_depth': 7},   0.7211736018908071


In [24]:
# RANDOM FOREST
rf_model = RandomForestClassifier()
tuner = Tuning(rf_model, param_grid_rf)
best_params_rf, best_score_rf = tuner.tune(X, y)

In [25]:
print(f"Best hyperparameters for random forests: {best_params_rf},   {best_score_rf}")

Best hyperparameters for random forests: {'max_depth': 11, 'n_estimators': 201},   0.723246805348265
